# Surrounding Landscape

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:




In [ ]:
import ee
import geemap
from utils import *

initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

biomass_raw = (ee.Image(f"projects/sat-io/open-datasets/ESA/ESA_CCI_AGB/CCI_BIOMASS_100m_AGB_{last_year}_v51").select("AGB").rename(f"ESA_CCI_{last_year}"))

# Load the categorical image and select the 'biome' band
biomes = ee.Image(f"{data_folder}/categorical").select("biome")
biomes_mask = biomes.eq(1).Or(biomes.eq(4)).rename("biome_mask")


## Surrounding mature forest biomass

- Mean biomass of surrounding mature forests (interpolated with Gaussian kernel)
- Total surrounding mature forest cover (normalized from 0-1) at a 500m distance

In [2]:
lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))
mature_mask = lulc.eq(3).reduce(ee.Reducer.allNonZero()).selfMask().updateMask(biomes_mask)
age = ee.Image(f"{data_folder}/mapbiomas_{last_year}")

mature_biomass = biomass_raw.updateMask(mature_mask).rename("mature_biomass")

# keep only forests that are 1500m away from any non-forest area
edge_detec = mature_mask.unmask(-1).zeroCrossing()
distance_forest_edge = (edge_detec.fastDistanceTransform(1000, 'pixels').sqrt() # 30m per pixel, 30 * 1000 = 30000m radius of search
    .multiply(ee.Image.pixelArea().sqrt()).toInt16().rename("dist"))

sur_cover = mature_mask.unmask(0).focalMean(radius = 500, units = "meters").float().rename("sur_cover")

In [3]:
distance_forest_edge = ee.Image("projects/amazon-forest-regrowth/assets/distance_forest_edge")

distance_gt_1000 = distance_forest_edge.gt(1000).selfMask()
mature_biomass_exclude_edge = mature_biomass.updateMask(distance_gt_1000)

edge_detec = mature_biomass_exclude_edge.unmask(-1).zeroCrossing()
distance_deep_forest = (edge_detec.fastDistanceTransform(1000, 'pixels').sqrt() # 30m per pixel, 30 * 1000 = 30000m radius of search
    .multiply(ee.Image.pixelArea().sqrt()).toInt16().rename("distance_deep_forest"))

## Biomass of nearest mature forest

In [4]:
# Aggregate the high-resolution pixels into the 10 km grid
mature_biomass_10k = mature_biomass_exclude_edge.updateMask(biomes_mask).reduceResolution(
    reducer = ee.Reducer.mean(),
    maxPixels = 1024,
    bestEffort = True # Use all pixels that can fit in the larger pixel
).reproject(
    crs = 'EPSG:4326',
    scale = 10000
).toInt16().rename("mature_biomass")

Export at a 100m resolution (same as ESA CCI Biomass) to argue for the distance-to-edge effect.

In [5]:
# export_image(distance_forest_edge, "distance_forest_edge", region = roi, scale = 30)
# export_image(sur_cover, "sur_cover", region = roi, scale = 30)
# export_image(mature_biomass, "mature_biomass", region = roi, scale = 100)
# export_image(mature_biomass_10k, "mature_biomass_10k", scale = 10000, region = roi)
# export_image(distance_deep_forest, "distance_deep_forest", region = roi, scale = 30)

Obtain the biomass of the nearest mature forests at 10k resolution. This will be used to build the asymptote of the model.

- Make a grid with the pixels in mature_biomass_10k that are within the biomes, but have NA values in mature_biomass_10k.
- For these pixels, obtain the distance and the value of the nearest mature forest pixel.
- Export the image with the values of the nearest mature forest pixel.

In [6]:
grid_amazon = ee.FeatureCollection(f"{data_folder}/grid_10k_amazon_non_mature_forests")

mature_biomass_10k = ee.Image("projects/amazon-forest-regrowth/assets/mature_biomass_10k")

edge_detec = mature_biomass_10k.unmask(-1).zeroCrossing()
distance_to_10k_forest = edge_detec.fastDistanceTransform(100, 'pixels').sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).add(10000).toInt32().rename("distance_to_10k_forest")

processed_fc = distance_to_10k_forest.reduceRegions(
    collection = grid_amazon,
    reducer = ee.Reducer.first(),
    scale = 10000
)

def buffer_feature(feature):
    distance = feature.getNumber('first').add(10000)
    buffer = feature.geometry().buffer(distance)
    return feature.setGeometry(buffer)

# Buffer each point to reach the nearest pixel
buffered_features = processed_fc.map(buffer_feature)

# Extract the biomass value for each buffered region
# This will get the value from nearest valid pixel
nearest_mature = mature_biomass_10k.reduceRegions(
    collection = buffered_features,
    reducer = ee.Reducer.firstNonNull(),
    scale = 10000,
    tileScale = 16
).map(lambda feature: feature.centroid())

nearest_mature_image = nearest_mature.reduceToImage(
    properties=['first'],
    reducer=ee.Reducer.first()
).unmask(0)

nearest_mature_image = nearest_mature_image.add(mature_biomass_10k.unmask(0)).selfMask().rename("nearest_mature")

# smooth it out to avoid sharp changes in expected biomass for remote areas far from mature forests
uniform_kernel = ee.Kernel.square(radius = 3, units = 'pixels')
nearest_mature_image = nearest_mature_image.reduceNeighborhood(reducer = ee.Reducer.mean(), kernel = uniform_kernel).rename("nearest_mature")

# export_image(nearest_mature_image, "nearest_mature", region = roi, scale = 10000)

## Amazon Quarters
Mean Biomass Value Per Climatic Region

In [7]:
# Load the mature biomass image
distance_gt_1000 = distance_forest_edge.gt(1000).selfMask()
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
mature_biomass_exclude_edge = mature_biomass.updateMask(distance_gt_1000).rename("mature_biomass")

# Load the individual feature collections and combine them into a single FeatureCollection
quarters = ee.FeatureCollection([
    ee.FeatureCollection(f"{data_folder}/raw/NE_outline_no_overlap").map(lambda f: f.set('sector', 1)),
    ee.FeatureCollection(f"{data_folder}/raw/NW_outline_no_overlap").map(lambda f: f.set('sector', 2)),
    ee.FeatureCollection(f"{data_folder}/raw/SW_outline_no_overlap").map(lambda f: f.set('sector', 3)),
    ee.FeatureCollection(f"{data_folder}/raw/SE_outline_no_overlap").map(lambda f: f.set('sector', 4)),
]).flatten()

ecoregions = (
    ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")
    .filterBounds(roi)
    .map(lambda feature: feature.intersection(roi))
)

# Function to get mean biomass per Feature
def create_biomass_image(feature):
    mean_biomass = mature_biomass_exclude_edge.reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry = feature.geometry(),
        scale = 500,
        maxPixels = 1e12
    )
    return feature.set(mean_biomass)

# Apply the function to each feature in the FeatureCollection
mean_biomass_quarters = quarters.map(create_biomass_image)
mean_biomass_ecoregions = ecoregions.map(create_biomass_image)

# Helper function to create an image from a property in the feature collection
def create_property_image(feature_collection, property_name):
    return feature_collection.reduceToImage(
        properties = [property_name],
        reducer = ee.Reducer.first()
    ).rename(property_name)

# Create images for the 'mature_biomass' and 'sector' properties
quarter_biomass = create_property_image(mean_biomass_quarters, 'mature_biomass').toInt16()
quarter = create_property_image(mean_biomass_quarters, 'sector').byte()
ecoreg_biomass = create_property_image(mean_biomass_ecoregions, 'mature_biomass').toInt16()

quarters_ecoreg_biomass = ee.Image.cat([quarter_biomass, quarter, ecoreg_biomass]).rename(["quarter_biomass", "quarter", "ecoreg_biomass"])

ecoreg = create_property_image(ecoregions, 'ECO_ID').toInt16().rename("ecoreg")

In [ ]:
# export_image(quarters_ecoreg_biomass, "quarters_ecoreg_biomass", region = roi, scale = 5000)
# export_image(ecoreg, "ecoreg", region = roi, scale = 1000)